In [2]:
#!pip install transformers

PIPELINE

In [4]:
from transformers import pipeline

classifier = pipeline('sentiment-analysis', )

sentence_positive = "I love cats"
sentence_negative = "I hate cats"

classifier(sentence_positive), classifier(sentence_negative)

Device set to use mps:0


([{'label': 'POSITIVE', 'score': 0.9995536208152771}],
 [{'label': 'NEGATIVE', 'score': 0.9943310022354126}])

MANUAL

In [23]:
#pip install pytorch-transformers

In [31]:
#from transformers import AutoTokenizer, AutoModelForSequenceClassification, DistilbertTokenizerFast
import torch
from transformers.models.distilbert import DistilBertForSequenceClassification
from transformers import AutoTokenizer

model_name = 'distilbert-base-uncased-finetuned-sst-2-english'
pt_model = AutoModelForSequenceClassification.from_pretrained(model_name)
#tokenizer = AutoTokenizer.from_pretrained(model_name) 
tokenizer_manual = AutoTokenizer.from_pretrained(model_name)

input_pos = tokenizer_manual(sentence_positive)
#input_neg = tokenizer_manual(sentence_negative)
input_neg_test = tokenizer_manual("I recklessly hate cats")

input_pos, input_neg, input_neg_test

loading configuration file config.json from cache at /Users/user/.cache/huggingface/hub/models--distilbert-base-uncased-finetuned-sst-2-english/snapshots/714eb0fa89d2f80546fda750413ed43d93601a13/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased-finetuned-sst-2-english",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "finetuning_task": "sst-2",
  "hidden_dim": 3072,
  "id2label": {
    "0": "NEGATIVE",
    "1": "POSITIVE"
  },
  "initializer_range": 0.02,
  "label2id": {
    "NEGATIVE": 0,
    "POSITIVE": 1
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.47.1",
  "vocab_size": 30522
}

loading weights file model.

({'input_ids': [101, 1045, 2293, 8870, 102], 'attention_mask': [1, 1, 1, 1, 1]},
 {'input_ids': [101, 1045, 5223, 8870, 102], 'attention_mask': [1, 1, 1, 1, 1]},
 {'input_ids': [101, 1045, 18555, 2135, 5223, 8870, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]})

In [30]:
tokenizer_manual.convert_ids_to_tokens([101, 1045, 2293, 8870, 102]), tokenizer_manual.convert_ids_to_tokens([101, 1045, 18555, 2135, 5223, 8870, 102])

(['[CLS]', 'i', 'love', 'cats', '[SEP]'],
 ['[CLS]', 'i', 'reckless', '##ly', 'hate', 'cats', '[SEP]'])

In [34]:
pt_batch = tokenizer_manual(
    [sentence_positive, sentence_negative, "I recklessly hate cats"],
    padding=True,
    truncation=True,
    max_length=512, 
    return_tensors='pt'
)

pt_batch


{'input_ids': tensor([[  101,  1045,  2293,  8870,   102,     0,     0],
        [  101,  1045,  5223,  8870,   102,     0,     0],
        [  101,  1045, 18555,  2135,  5223,  8870,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1]])}

In [35]:
pt_outputs = pt_model(**pt_batch)
pt_outputs

SequenceClassifierOutput(loss=None, logits=tensor([[-3.7413,  3.9726],
        [ 2.8008, -2.3663],
        [ 2.5925, -2.1713]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [36]:
from torch import nn
pt_predictions = nn.functional.softmax(pt_outputs.logits, dim=-1)

pt_predictions

tensor([[4.4638e-04, 9.9955e-01],
        [9.9433e-01, 5.6690e-03],
        [9.9154e-01, 8.4607e-03]], grad_fn=<SoftmaxBackward0>)